# LangChain: Models, Prompts and Output Parsers


## Outline

 * Direct API calls to OpenAI
 * API calls through LangChain:
   * Prompts
   * Models
   * Output parsers

## EITHER: get your [OpenAI API Key](https://platform.openai.com/account/api-keys)

In [ ]:
#!pip install python-dotenv
#!pip install openai

In [4]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']
# openai.api_key = ""

In [15]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_name = "gpt-3.5-turbo"
else:
    llm_name = "gpt-3.5-turbo-0301"

## OR: use [LocalAI as an OpenAI replacement](https://localai.io/howtos/easy-request-openai/)

In [1]:
import os
import openai

# Specify the port your LocalAI docker container runs on
openai.api_base = "http://localhost:8080/v1"
openai.api_key = "sx-xxx"  # not needed for LocalAI (dummy)
OPENAI_API_KEY = "sx-xxx"
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [3]:
# Specify the model you are using
# llm_name = ""
llm_name = "lunademo"

## Chat API : OpenAI

Let's start with a direct API calls to OpenAI.</br>
In this example we write a custom function to do what we can later more easily achieve with LangChain (see below).

In [6]:
def get_completion(prompt, model=llm_model):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0.7,
    )
    return response.choices[0].message["content"]


In [7]:
get_completion("I have 5 bananas and I eat 3. I also get another 1 as a present. How many do I have left?")

'You would have 3 bananas left.'

In [8]:
customer_email = """
Aaaah!!!! I need an OpenAI API key to run this. \
I got one with $18 Credits back in March '23, \
but the credits expired without me using them! \
And now OpenAI won't let me create a new account, \
because my phone number has already been used! \
And I don't have a credit card, \
to pay for a subscription!!! \
So we need to find an Open Source model!!!
"""

In [9]:
style = """Pirate English \
in a relaxed tone"""

In [10]:
prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

Translate the text that is delimited by triple backticks 
into a style that is Pirate English in a relaxed tone.
text: ```
Aaaah!!!! I need an OpenAI API key to run this. I got one with $18 Credits back in March '23, but the credits expired without me using them! And now OpenAI won't let me create a new account, because my phone number has already been used! And I don't have a credit card, to pay for a subscription!!! So we need to find an Open Source model!!!
```



In [11]:
response = get_completion(prompt)

In [12]:
response

"Arrrr!!!! Me hearties, me be needin' an OpenAI API key to run this here program. I did manage to get one wit' $18 Credits back in March '23, but the credits expired without me usin' 'em! And now OpenAI won't let me create a new account, because me phone number has already been used! And I don't have a credit card, to pay for a subscription!!! So we need to find an Open Source model!!!"

## Output

> Uff!!! I bruche en OpenAI API-Schlüssel, um das z'laufe. Ich ha im März '23 en mit $18 Guthabe übercho, aber s Guthabe isch verfalle ohni dass ich's brucht ha! Und jetzt laht mich OpenAI kei neus Konto erstelle, will mini Telefonnummer scho benutzt worde isch! Und ich ha kei Kreditkart, um für es Abonnement z'zahle!!! Mir müend also en Open Source Modell finde!!!

## Chat API : LangChain

Let's try how we can do the same using LangChain.

In [ ]:
#!pip install --upgrade langchain

### Model

In [16]:
from langchain.chat_models import ChatOpenAI

### Prompt template

In [18]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [19]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)


In [20]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n')

In [21]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [22]:
customer_style = """Caribbean Pirate English \
in a relaxed tone
"""

In [23]:
customer_email = """
Aaaah!!!! I need an OpenAI API key to run this. \
I got one with $18 Credits back in March '23, \
but the credits expired without me using them! \
And now OpenAI won't let me create a new account, \
because my phone number has already been used! \
And I don't have a credit card, \
to pay for a subscription!!! \
So we need to find an Open Source model!!!
"""

In [24]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [25]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain.schema.messages.HumanMessage'>


In [26]:
# Call the LLM to translate to the style of the customer message
customer_response = chat(customer_messages)

In [27]:
print(customer_response.content)

Arrr!!!! Ye scallywags! Me be needin' an OpenAI API key t' run this here program. Back in March '23, I found me one that came with $18 Credits, but the looting scoundrels expired without me gettin' to use 'em! Now OpenAI won't be lettin' me create a new account, claimin' that me phone number's already been used. And to make matters worse, I don't be havin' a credit card to pay for a subscription! So, me hearties, we be needin' t' find an Open Source model - savvy?


In [28]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [29]:
service_style_pirate = """\
a impolite tone \
that speaks in Pirate\
"""

In [30]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

Translate the text that is delimited by triple backticks into a style that is a impolite tone that speaks in Pirate. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
```



In [31]:
service_response = chat(service_messages)
print(service_response.content)

Arrgghh matey! Listen here ye scurvy customer, the blasted warranty be not coverin' yer grog-infused cleaning expenses for yer kitchen. It be yer own fault that ye misused the infernal blender by forgettin' to put the cursed lid on before startin' it up. So tough luck and walk the plank! Farewell, ye landlubber!


## Output Parsers

Let's start with defining how we would like the LLM output to look like:

In [32]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [33]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

**Import template**

In [34]:
from langchain.prompts import ChatPromptTemplate

**Create prompt templates from review template**

In [35]:
prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'))]


**Create messages to pass to OpenAI API endpoint**

In [36]:
messages = prompt_template.format_messages(text=customer_review)

**Create OpenAI endpoint**

In [38]:
chat = ChatOpenAI(
    temperature=0.0, 
    model=llm_name,
    # openai_api_key=""
)

**Call endpoint and get response**

In [39]:
response = chat(messages)
print(response.content)

{
    "gift": true,
    "delivery_days": 2,
    "price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}


**Note that the response should have been a JSON, but is actually just a string which looks like a JSON**

In [40]:
type(response.content)

str

In [ ]:
# You will get an error by running this line of code 
# because'gift' is not a dictionary
# 'gift' is a string
response.content.get('gift')

### Parse the LLM output string into a Python dictionary

In [42]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

**Specify the schema for each variable using langchain ResponseSchema**

In [43]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

In [44]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [45]:
format_instructions = output_parser.get_format_instructions()

In [46]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```


In [47]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

In [48]:
print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```

In [49]:
response = chat(messages)

In [50]:
print(response.content)

```json
{
	"gift": true,
	"delivery_days": "2",
	"price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}
```


**Still a string that looks like a dict**

In [ ]:
{
    "gift": true,
    "delivery_days": 2,
    "price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}

**Therefore, use the parser**

In [52]:
output_dict = output_parser.parse(response.content)

In [53]:
output_dict

{'gift': True,
 'delivery_days': '2',
 'price_value': ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]}

In [54]:
{'gift': True,
 'delivery_days': '2',
 'price_value': ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]}

{'gift': True,
 'delivery_days': '2',
 'price_value': ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]}

**This is now actually of type(dict)**

In [55]:
type(output_dict)

dict

In [56]:
output_dict.get('delivery_days')

'2'

### Source: https://learn.deeplearning.ai/langchain/lesson/2/models,-prompts-and-parsers